In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (16901 Collins Ave #5305, Sunny Isles Beach, F...
2     (5313 Fisher Island Dr #5313, Miami, FL 33109,...
3     (360 Ocean Dr #1101s, Key Biscayne, FL 33149, ...
4     (2675 S Bayshore Dr #1201s, Miami, FL 33133, U...
5     (100 South Pointe Dr APT 3106, Miami Beach, FL...
6     (789 Crandon Blvd PH 6, Key Biscayne, FL 33149...
7     (102 24th St, Miami Beach, FL 33139, USA, (25....
8     (18975 Collins Ave #4602, Sunny Isles Beach, F...
9     (2821 S Bayshore Dr #10b, Miami, FL 33133, USA...
10    (360 Ocean Dr #604s, Key Biscayne, FL 33149, U...
Name: loc, dtype: object

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Jade Signature\r16901 Collins Ave 5305\rSunny ...,4/29/2019,6/1/2022,1060,"$10,300,000.00",7916,"$1,582.42",Sandra Chartouni,Fortune International Realty,Daniel Alitenssi,Prime International Assoc Inc,Jade Signature 16901 Collins Ave 5305 Sunny Is...,Jade Signature,16901 Collins Ave 5305 Sunny Isles Beach,"(16901 Collins Ave #5305, Sunny Isles Beach, F...","(25.9327557, -80.1210054, 0.0)",25.932756,-80.121005,0.0
2,Bayview\r5313 Fisher Island Dr 5313\rMiami Beach,12/22/2021,6/1/2022,65,"$10,000,000.00",6209,"$1,610.57",Karla Abaunza,Luxury Living Realty,Karla Abaunza,Luxury Living Realty,Bayview 5313 Fisher Island Dr 5313 Miami Beach,Bayview,5313 Fisher Island Dr 5313 Miami Beach,"(5313 Fisher Island Dr #5313, Miami, FL 33109,...","(25.7636181, -80.1486308, 0.0)",25.763618,-80.148631,0.0
3,Oceana Key Biscayne\r360 Ocean Dr 1101S\rKey B...,2/8/2022,5/31/2022,44,"$8,500,000.00",3791,"$2,242.15",Riley Smith,BHHS EWM Realty,Riley Smith,BHHS EWM Realty,Oceana Key Biscayne 360 Ocean Dr 1101S Key Bis...,Oceana Key Biscayne,360 Ocean Dr 1101S Key Biscayne,"(360 Ocean Dr #1101s, Key Biscayne, FL 33149, ...","(25.6942877, -80.1578218, 0.0)",25.694288,-80.157822,0.0
4,The Grove at Grand Bay\r2675 S Bayshore Dr 120...,3/16/2022,5/31/2022,16,"$8,400,000.00",4824,"$1,741.29",John De Olazarra,Properties Group Inc,Michael Martirena,"Compass Florida, LLC",The Grove at Grand Bay 2675 S Bayshore Dr 1201...,The Grove at Grand Bay,2675 S Bayshore Dr 1201S Miami,"(2675 S Bayshore Dr #1201s, Miami, FL 33133, U...","(25.729886, -80.2368583, 0.0)",25.729886,-80.236858,0.0
5,Continuum on South Beach\r100 S Pointe Dr 3106...,5/2/2022,5/31/2022,9,"$7,050,000.00",2048,"$3,442.38",Juan Alvarez,Blokhaus Real Estate,Sheila Rojas,Brown Harris Stevens,Continuum on South Beach 100 S Pointe Dr 3106 ...,Continuum on South Beach,100 S Pointe Dr 3106 Miami Beach,"(100 South Pointe Dr APT 3106, Miami Beach, FL...","(25.7676323, -80.137348, 0.0)",25.767632,-80.137348,0.0
6,Club Tower One\r789 Crandon Blvd PH-6\rKey Bis...,2/24/2022,6/1/2022,11,"$6,850,000.00",2850,"$2,403.51",Claudine Coto,One Sotheby's Intl Realty,Darin Spencer Raymond PA,BHHS EWM Realty,Club Tower One 789 Crandon Blvd PH-6 Key Biscayne,Club Tower One,789 Crandon Blvd PH-6 Key Biscayne,"(789 Crandon Blvd PH 6, Key Biscayne, FL 33149...","(25.6868145, -80.15900479999999, 0.0)",25.686815,-80.159005,0.0
7,1 Hotel & Homes\r102 24th St 1520\rMiami Beach,10/1/2021,6/3/2022,189,"$5,100,000.00",1386,"$3,679.65",Jennifer Restrepo,EXP Realty LLC,Jennifer Restrepo,EXP Realty LLC,1 Hotel & Homes 102 24th St 1520 Miami Beach,,1 Hotel & Homes 102 24th St 1520 Miami Beach,"(102 24th St, Miami Beach, FL 33139, USA, (25....","(25.7991784, -80.1272173, 0.0)",25.799178,-80.127217,0.0
8,Residences by Armani Casa\r18975 Collins Ave 4...,1/28/2021,5/31/2022,449,"$4,550,000.00",2682,"$1,696.50",Mariana Niro,Douglas Elliman,Christian Espinosa,Optimar International Realty,Residences by Armani Casa 18975 Collins Ave 46...,Residences by Armani Casa,18975 Collins Ave 4602 Sunny Isles Beach,"(18975 Collins Ave #4602, Sunny Isles Beach, F...","(25.9529783, -80.1191546, 0.0)",25.952978,-80.119155,0.0
9,Two Park Grove\r2821 S Bayshore Dr 10B\rMiami,2/7/2022,6/1/2022,8,"$3,999,000.00",2532,"$1,579.38",Pietro Belmonte,Douglas Elliman,Rose Harris,One Sotheby's Intl Realty,Two Park Grove 2821 S Bayshore Dr 10B Miami,Two Park Grove,2821 S Bayshore Dr 10B Miami,"(2821 S Bayshore Dr #10b, Miami, FL 33133, USA...","(25.7283709, -80.238272, 0.0)",25.728371,-80.238272,0.0
10,O0 ceana Key Biscayne\r360 Ocean Dr 604S\rKey ...,1/25/2022,5/31/2022,82,"$3,960,000.00",1703,"$2,325.31",Andrey Jimenez-Trushina,One Sotheby's Intl Realty,Nurit Edelman,"Nurit Edelman Realty, Inc",O0 ceana Key Biscayne 360 Ocean Dr 604S Key Bi...,O,0 ceana Key Bisca

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [11]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [10]:
df.at[10,'building_name']=('Oceana Key Biscayne')
df.at[10,'address_only']=('360 Ocean Dr 604S Key Biscayne')

In [11]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [12]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('May 29th - June 4th')

marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [13]:
m.save('index.html')